<p style="width:35%;float:right;padding-left:50px">
<img src=http://www.geophysik.uni-muenchen.de/~igel/snap.png?raw=true>
<span style="font-size:smaller">
from <span style="font-style: italic">Nissen-Meyer et al. (2014). AxiSEM: broadband 3-D seismic wavefields in axisymmetric media. Solid Earth, 5(1), 425–445.</span>
</span>
</p>

# Computational Seismology 


## *Spectral Element Method Elastic Wave Equation in 1D*


**Authors:** *Heiner Igel, Lion Krischer, Florian Wölfl, Stephanie Wollherr*

This exercise animates the Elastic Wave Equation in 1D with the Spectral Element Method (SEM): <br>

\begin{eqnarray*}
\rho(x) \partial_t^2 u(x,t) + \partial_x (\mu(x) \partial_x u(x,t)) = 0
\end{eqnarray*}


<div style="clear:both"></div>

**Exercises:**
1. Read the code and try to understand how the mass and stiffness matrix are initialized elementwise and later globally. <br>



2. Use this 1D spectral element code **SEM1D** to determine experimentally the stability limit as a function of the order of the scheme. Increase the order of the scheme and observe the necessary decrease of the time step, when the Courant criterion is kept constant (There are some hints inside the code where the Courant criterium is calculated).
<br>

***For spectral elements experts***
3. Modify **SEM1D** to allow for space-dependent elastic parameters. Introduce a low-velocity zone (-30%) at the center of the model spanning 5 elements. 

4. Introduce $h-$adaptivity to the numerical scheme by making the Jacobian element dependent. Generate a space-dependent mesh size (e.g., decreasing the element size gradually towards the centre). Generate a velocity model that keeps the number of points per wavelength approximately constant. 

In [ ]:
# This is a configuration step for the exercise. Please run it before the simulation code!
# Imports of python modules 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from operator import sub

from gll import gll
from lagrange1st import lagrange1st 
from legendre import legendre 
from ricker import ricker 
import time

# Show the plots in the Notebook.
plt.switch_backend("nbagg")

We approximate (i.e., ”interpolate”) our unknown displacement field u(x, t) by a sum over space-dependent basis functions $\psi_i$ weighted by time-dependent coefficients $u_i(t)$

$$ u(x,t) \ \approx \ \overline{u}(x,t) \ = \ \sum_{i=1}^{n} u_i(t) \ \psi_i(x) $$

As interpolating functions we choose the Lagrange polynomials and use $\xi$ as the space variable representing our elemental domain:

$$ \psi_i \ \rightarrow \ \ell_i^{(N)} (\xi) \ := \ \prod_{k = 1, \ k \neq i}^{N+1} \frac{\xi - \xi_k}{\xi_i-\xi_k}, \qquad   i = 1, 2, \dotsc , N + 1  $$
They are implemented in Python with the following code:

In [ ]:
def lagrange(N,i,x):

# Program to calculate  Lagrange polynomial for order N
# and polynomial i [0, N] at location x

    from gll import gll

    [xi, weights] =  gll(N)
    fac = 1
    for j in range (-1,N): 
        if j != i:
            fac=fac*((x-xi[j+1])/(xi[i+1]-xi[j+1]))
    x = fac

    return x

<p style="width:50%;float:right;padding-left:50px">
<img src=fig_sem_symbolic.png>
<span style="font-size:smaller">
</span>
</p>

Now we have to initialize the mass and stiffness matrix at the element level. <br>
The elemental <b> mass matrix </b> is defined as

$$ M_{ji}^e \ = \ w_j  \ \rho (\xi)  \ \frac{\mathrm{d}x}{\mathrm{d}\xi} \delta_{ij} \vert_ {\xi = \xi_j}   $$

and the elemental <b> stiffness matrix </b> as

$$ K_{ji}^e \ = \ \sum_{k = 1}^{N+1} w_k \ \mu (\xi)  \ \partial_\xi \ell_j (\xi) \ \partial_\xi \ell_i (\xi) \left(\frac{\mathrm{d}\xi}{\mathrm{d}x} \right)^2 \frac{\mathrm{d}x}{\mathrm{d}\xi} \vert_{\xi = \xi_k} $$
                                                                                                                                                                                           

Later we combine them to global matrices.

In [ ]:
# Initialization of setup
# ---------------------------------------------------------------
nt = 10000          # number of time steps
xmax = 10000.       # Length of domain
vs = 2500.          # [m/s] S velocity for homogeneneous medium
rho = 2000          # [kg/m3] Density for homogeneous model
mu = rho * vs**2    # Initialization of shear modulus mu
N = 3               # Order of Lagrange polynomials THIS WAS ORIGINALLY 5
ne = 250            # Number of elements
Tdom = .2           # Dominant period of Ricker source wavelet
iplot = 20          # Plotting each iplot snapshot

# variables for elemental matrices
Me = np.zeros(N+1, dtype =  float)
Ke = np.zeros([N+1, N+1], dtype =  float)

# ----------------------------------------------------------------

# Initialization of GLL points integration weights
[xi, w] = gll(N)    # xi -> N+1 coordinates [-1 1] of GLL points
                    # w Integration weights at GLL locations
# Space domain
le = xmax/ne       # Length of elements


# Vector with GLL points  
k=0
xg = np.zeros((N*ne)+1) 
xg[k] = 0
for i in range(1,ne+1):
    for j in range(0,N):
        k = k+1
        xg[k] = (i-1)*le+.5*(xi[j+1]+1)*le

# ---------------------------------------------------------------
# Calculation if time step is according to Courant criterion
# here we fixed the Courant value, for exercise 1 choose a fixed maximum timestep dt=0.00018 and 
# calculate the corresponding eps transforming the formula below
# how does it evolve when you increase the order? (print it)

dxmin = min(np.diff(xg))  
eps = 0.1           # Courant value
dt = eps*dxmin/vs   # Global time step

print ('The Courant value for order %g is %g' %(N, eps))

# Mapping - Jacobian
J = le/2 
Ji = 1/J             # Inverse Jacobian

# Initialization of 1st derivative of Lagrange polynomials
l1d = lagrange1st(N)   # Array with GLL as columns for each N+1 polynomial

# -----------------------------------------------------------------
# Initialization of system matrices
# -----------------------------------------------------------------
# Elemental Mass matrix
for i in range(-1,N):
    Me[i+1]=rho*w[i+1]*J #stored as a vector since it's diagonal
    
print('This are the diagonal entries of the Elemental Mass matrix: ')
print(Me)
print('\n')

# Global Mass matrix
k=-1
ng=(ne-1)*N+N+1
M=np.zeros(2*ng) 

for i in range(1, ne+1):  
    for j in range(0, N+1): 
        k=k+1
        if i>1:
            if j==0:
                k=k-1

        M[k]=M[k]+Me[j]

# Build inverse matrix (this is overkill, but just to get the complete
# matrix system for tutorial purposes)
Minv = np.identity(ng)
for i in range(0,ng):
    Minv[i,i]=1./M[i]
    
# ---------------------------------------------------------------
# Elemental Stiffness Matrix

for i in range(-1,N):
    for j in range(-1,N):
            sum=0
            for k in range(-1,N):
                sum = sum + mu*w[k+1]*Ji**2 *J*l1d[i+1,k+1]*l1d[j+1,k+1]

            Ke[i+1,j+1]=sum

print('This is the elemental stiffness matrix:')
print(Ke)
            
# Global Stiffness Matrix
K = np.zeros([ng, ng])

# Values except at element boundaries 
for k in range(1,ne+1):
    i0=(k-1)*N+1
    j0=i0
    for i in range(-1,N):
        for j in range(-1,N):
            K[i0+i,j0+j]=Ke[i+1,j+1]



# Values at element boundaries 
for k in range(2,ne+1):
    i0=(k-1)*N
    j0=i0
    K[i0,j0]=Ke[0,0]+Ke[N,N]

    
# initialize source time function and force vector f
src = ricker(dt,Tdom)

# Plot of the source time function
plt.figure()
plt.plot(np.arange(1,len(src)+1) * dt, src)
plt.xlabel(' Time (s)')
plt.ylabel('Amplitude')
plt.title(' Source time function ')
plt.show() 
isrc= np.floor(ng/2)   # Source location

In [ ]:
# Initialization of solution vectors
u = np.zeros(ng)
uold = u
unew = u
f = u 

################ Time extrapolation ####################################
# Simple finite difference time extrapolation
  
# Interactive mode on for plotting
# plt.close()
plt.figure()
plt.ion()

plt.title('SEM 1D Animation')
plt.xlabel(' x (m)')
plt.ylabel(' Amplitude ')

# Initialize animated plot
lines = plt.plot(xg, u, lw=1.5)
plt.show()

for it in range(nt): 
    # Source initialization
    f= np.zeros(ng)
    if it < len(src):
        f[isrc-1] = src[it-1] 
            
    
    # Time extrapolation: Euler scheme
    unew = dt**2 * np.dot(Minv, f - np.dot(K, u)) + 2 * u - uold
    
    uold = u
    u = unew
    
 
    # Animation plot
    if not it % iplot:
        for l in lines:
            l.remove()
            del l
        lines = plt.plot(xg, u, color="black", lw = 1.5)
        plt.draw()
        # time.sleep(0.1)   # to slow the animation down
        # print('it = ', it, ', Max(u) = ', max(u))  # Print the timestep and the current amplitude